In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,MinMaxScaler,LabelEncoder
import torch
from torch.utils.data import Dataset, DataLoader, Subset, random_split
from torchvision import transforms
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as f
from PIL import Image
import os
from sklearn import metrics
import glob
 

In [2]:
df=pd.read_csv("data_imu_birlestirilmis.csv")

In [3]:
label_encoder=LabelEncoder()

In [4]:
df["Class Name"]=label_encoder.fit_transform(df["Class Name"])

In [5]:
print(label_encoder.classes_)



['falling' 'not_falling']


In [6]:
df

,Unnamed: 0,ax,ay,az,gx,gy,gz,timestamp,Class Name
0,0,1.091892,8.707838,1.765225,-0.036396,0.063694,-0.036396,1900-01-01 17:44:42.000000,1
1,1,1.083568,8.732087,1.765318,-0.036781,0.064346,-0.036686,1900-01-01 17:44:42.020000,1
2,2,1.075247,8.756325,1.765411,-0.037166,0.064996,-0.036976,1900-01-01 17:44:42.040000,1
3,3,1.066932,8.780539,1.765503,-0.037551,0.065642,-0.037265,1900-01-01 17:44:42.060000,1
4,4,1.058625,8.804720,1.765596,-0.037936,0.066281,-0.037553,1900-01-01 17:44:42.080000,1
...,...,...,...,...,...,...,...,...,...
51645,51645,1.611913,8.047413,-3.202833,-0.106623,0.084735,-0.050371,1900-01-01 17:59:15.899794,1
51646,51646,1.626502,8.043946,-3.201882,-0.118920,0.123886,-0.071806,1900-01-01 17:59:15.919794,1
51647,51647,1.641042,8.040466,-3.200937,-0.131261,0.163164,-0.093307,1900-01-01 17:59:15.939794,1
51648,51648,1.655544,8.036974,-3.199998,-0.143635,0.202538,-0.114858,1900-01-01 17:59:15.959793,1


In [7]:
dataframe=df

In [8]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51650 entries, 0 to 51649
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  51650 non-null  int64  
 1   ax          51650 non-null  float64
 2   ay          51650 non-null  float64
 3   az          51650 non-null  float64
 4   gx          51650 non-null  float64
 5   gy          51650 non-null  float64
 6   gz          51650 non-null  float64
 7   timestamp   51650 non-null  object 
 8   Class Name  51650 non-null  int32  
dtypes: float64(6), int32(1), int64(1), object(1)
memory usage: 3.3+ MB


import torch
from torch.utils.data import Dataset
from scipy import stats

class CustomDataset(Dataset):
    def __init__(self, dataframe, sequence_length=25):
        self.dataframe = dataframe
        self.features = ["ax", "ay", "az", "gx", "gy", "gz"]
        self.labels = "Class Name"
        self.sequence_length = sequence_length

        self.scaler = StandardScaler()
        self.dataframe[self.features] = self.scaler.fit_transform(self.dataframe[self.features])

    def __len__(self):
        return len(self.dataframe) - self.sequence_length + 1

    def __getitem__(self, idx):
         
        features_sequence = self.dataframe.loc[idx:idx + self.sequence_length - 1, self.features].values.astype(float)
        
        
        label = self.dataframe.loc[idx + self.sequence_length - 1, self.labels]
        
         
        features_tensor = torch.tensor(features_sequence, dtype=torch.float32)
        label_tensor = torch.tensor(label, dtype=torch.long)

        return features_tensor, label_tensor

    

In [12]:
import torch
from torch.utils.data import Dataset
from sklearn.preprocessing import StandardScaler

class CustomDataset(Dataset):
    
    def __init__(self, dataframe, sequence_length=25):
        self.dataframe = dataframe
        self.features = ["ax", "ay", "az", "gx", "gy", "gz"]
        self.labels = "Class Name"
        self.sequence_length = sequence_length

        self.scaler = StandardScaler()
        self.dataframe[self.features] = self.scaler.fit_transform(self.dataframe[self.features])
    
    def __len__(self):
        return len(self.dataframe) - self.sequence_length + 1
    
    def __getitem__(self, idx):
        while True:
            features_sequence = self.dataframe.loc[idx:idx + self.sequence_length - 1, self.features].values.astype(float)
            labels_sequence = self.dataframe.loc[idx:idx + self.sequence_length - 1, self.labels].values

            # Tüm etiketlerin aynı olup olmadığını kontrol et
            if len(set(labels_sequence)) == 1:  # Tüm etiketler aynı
                label = labels_sequence[0]
                break  # Aynı etiket bulundu, döngüden çık

            # Farklı etiketler varsa, bir sonraki indeksi kontrol et
            idx += 1
            if idx + self.sequence_length > len(self.dataframe):  # Sınır kontrolü
                raise IndexError("Veri setinde uygun pencere kalmadı.")

        features_tensor = torch.tensor(features_sequence, dtype=torch.float32)  # (sequence_length, features_count)
        label_tensor = torch.tensor(label, dtype=torch.long)

        return features_tensor, label_tensor

# standard scale önemli

In [13]:
dataset = CustomDataset(dataframe)

 
labels = dataset.dataframe['Class Name'].values[:len(dataset)]   

 
indices = list(range(len(dataset)))

 
train_indices, test_indices = train_test_split(indices, test_size=0.2, stratify=labels)
 
train_dataset = torch.utils.data.Subset(dataset, train_indices)
test_dataset = torch.utils.data.Subset(dataset, test_indices)

 
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [14]:
print(f"Train dataset size: {len(train_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")

Train dataset size: 41300
Test dataset size: 10326


In [15]:
#torch.cuda.empty_cache()

In [16]:
train_dataset[6]

(tensor([[ 0.3258,  0.0611,  0.9339, -0.4327, -1.0874, -1.9737],
         [ 0.3341,  0.0286,  0.9693, -0.4955, -1.0221, -1.9335],
         [ 0.3423, -0.0049,  1.0047, -0.5595, -0.9573, -1.8904],
         [ 0.3504, -0.0392,  1.0400, -0.6245, -0.8928, -1.8448],
         [ 0.3583, -0.0744,  1.0751, -0.6904, -0.8287, -1.7968],
         [ 0.3662, -0.1103,  1.1100, -0.7570, -0.7652, -1.7465],
         [ 0.3739, -0.1469,  1.1446, -0.8242, -0.7022, -1.6940],
         [ 0.3816, -0.1841,  1.1791, -0.8918, -0.6398, -1.6396],
         [ 0.3892, -0.2217,  1.2132, -0.9596, -0.5780, -1.5834],
         [ 0.3967, -0.2599,  1.2471, -1.0274, -0.5169, -1.5255],
         [ 0.4041, -0.2984,  1.2806, -1.0951, -0.4565, -1.4662],
         [ 0.4115, -0.3372,  1.3138, -1.1626, -0.3968, -1.4055],
         [ 0.4188, -0.3762,  1.3465, -1.2296, -0.3380, -1.3436],
         [ 0.4260, -0.4154,  1.3789, -1.2960, -0.2801, -1.2807],
         [ 0.4332, -0.4547,  1.4108, -1.3616, -0.2230, -1.2169],
         [ 0.4403, -0.493

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [18]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout_prob):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout_prob)
        self.fc = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        h_0 = torch.zeros(num_layers, x.size(0), hidden_size).to(device)   
        c_0 = torch.zeros(num_layers, x.size(0), hidden_size).to(device)   
        
        out, _ = self.lstm(x, (h_0, c_0))  # LSTM forward
        out = self.fc(out[:, -1, :])  # Son zaman adımının çıktısını alır
        out = self.sigmoid(out)  # Sigmoid ile iki sınıfa ayırır
        return out

In [19]:
input_size = 6  # ax, ay, az, gx, gy, gz (6 özellik)
hidden_size = 24 # Gizli katman boyutu
num_layers = 2  # LSTM katman sayısı
output_size = 1  # Binary classification
dropout_prob = 0.5  # Dropout oranı


In [20]:
model = LSTMModel(input_size, hidden_size, num_layers, output_size, dropout_prob).to(device)
criterion = nn.BCELoss()   
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [21]:
model.to(device)

LSTMModel(
  (lstm): LSTM(6, 24, num_layers=2, batch_first=True, dropout=0.5)
  (fc): Linear(in_features=24, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

for features,labels in train_loader:
        features.labels=features.to(device),labels.to(device)
        print(features[0])

In [22]:
num_epochs=30
for epoch in range(num_epochs):
    model.train()
    for features,labels in train_loader:
        features,labels=features.to(device),labels.to(device)
        optimizer.zero_grad()
        outputs =model(features)
        outputs = torch.squeeze(outputs)
        labels = labels.float()
        loss=criterion(outputs,labels)
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

print("Training complete") 

Epoch [1/30], Loss: 0.4074
Epoch [2/30], Loss: 0.3831
Epoch [3/30], Loss: 0.3751
Epoch [4/30], Loss: 0.4785
Epoch [5/30], Loss: 0.0717
Epoch [6/30], Loss: 0.0890
Epoch [7/30], Loss: 0.6156
Epoch [8/30], Loss: 0.2453
Epoch [9/30], Loss: 0.1239
Epoch [10/30], Loss: 0.0703
Epoch [11/30], Loss: 0.1962
Epoch [12/30], Loss: 0.1446
Epoch [13/30], Loss: 0.0746
Epoch [14/30], Loss: 0.0079
Epoch [15/30], Loss: 0.0899
Epoch [16/30], Loss: 0.0651
Epoch [17/30], Loss: 0.0727
Epoch [18/30], Loss: 0.0524
Epoch [19/30], Loss: 0.1302
Epoch [20/30], Loss: 0.1158
Epoch [21/30], Loss: 0.1043
Epoch [22/30], Loss: 0.1362
Epoch [23/30], Loss: 0.0383
Epoch [24/30], Loss: 0.1839
Epoch [25/30], Loss: 0.2975
Epoch [26/30], Loss: 0.0191
Epoch [27/30], Loss: 0.0095
Epoch [28/30], Loss: 0.3149
Epoch [29/30], Loss: 0.0500
Epoch [30/30], Loss: 0.1176
Training complete


In [23]:
model.eval()   
test_loss = 0.0
correct_predictions = 0
total_predictions = 0

with torch.no_grad():   
    for features, labels in test_loader:
        features, labels = features.to(device), labels.to(device)
        
        outputs = model(features)
        outputs = torch.squeeze(outputs) 
        labels = labels.float()
        loss = criterion(outputs, labels)
        test_loss += loss.item()

        # Tahminleri yap
        predicted = (outputs > 0.5).float()
        correct_predictions += (predicted == labels).sum().item()
        total_predictions += labels.size(0)

 
average_test_loss = test_loss / len(test_loader)
accuracy = correct_predictions / total_predictions

print(f'Test Loss: {average_test_loss:.4f}, Accuracy: {accuracy:.4f}')

Test Loss: 0.0469, Accuracy: 0.9829


In [24]:
 
torch.save(model.state_dict(), 'lstm_model__23_09_2024.pth')


In [ ]:
 
model = LSTMModel(input_size, hidden_size, num_layers, output_size, dropout_prob).to(device)
 
model.load_state_dict(torch.load('lstm_model_23_09_.pth'))
